In [155]:
import numpy as np
import scipy as sp 
import pandas as pd
from scipy import stats as sps
from scipy import optimize
from sklearn.linear_model import LogisticRegression
from IPython.display import display, Markdown

In [2]:
from datetime import datetime

## Example data from Chapter 2 of "What If?"

In [3]:
ch2data = {
    'L': [0]*8 + [1]*12,
    'A': [0]*4 + [1]*4 + [0]*3 + [1]*9,
    'Y': [0,1] + [0]*5 + [1]*3 + [0] + [1]*6 + [0]*3,
}
ch2names = ['Rheia', 'Kronos', 'Demeter', 'Hades', 'Hestia', 'Poseidon', 'Hera', 
            'Zeus', 'Artemis', 'Apollo', 'Leto', 'Ares', 'Athena', 'Hephaestus', 
            'Aphrodite', 'Cyclope', 'Persephone', 'Hermes', 'Hebe', 'Dionysus']
pd.DataFrame(ch2data, index=ch2names)

,L,A,Y
Rheia,0,0,0
Kronos,0,0,1
Demeter,0,0,0
Hades,0,0,0
Hestia,0,1,0
Poseidon,0,1,0
Hera,0,1,0
Zeus,0,1,1
Artemis,1,0,1
Apollo,1,0,1


## Basic function definitions

A utility function for generating all binary vectors of a given length.

In [4]:
def all_binary_vectors(length):
    result = [[]]
    while len(result[0]) < length:
        result = [ vector + [val] for vector in result for val in [False, True] ]
    return result

In [5]:
pd.DataFrame(all_binary_vectors(3))

,0,1,2
0,False,False,False
1,False,False,True
2,False,True,False
3,False,True,True
4,True,False,False
5,True,False,True
6,True,True,False
7,True,True,True


Calculate the standardized means for each level of A for a given dataset

In [6]:
def fit_phat_Y_given_L_A(L, A, Y):
    A = np.array(A)
    Y = np.array(Y)
    L = np.reshape(np.array(L), (len(Y), -1)) # force L to be a 2D array
    M = L.shape[1]
    A_levels = [False, True]
    L_levels = all_binary_vectors(M)
    phat_Y_given_L_A_table = np.empty((len(L_levels), len(A_levels)))
    
    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        for j,a in enumerate(A_levels):
            phat_Y_given_L_A_table[i,j] = np.mean(Y[mask_l * (A == a)])
    
    def phat_Y_given_L_A(L, A):
        A = np.array(A)
        L = np.reshape(np.array(L), (-1, M)) # force L to be a 2D array
        result = np.zeros(L.shape[0])
        for i,l in enumerate(L_levels):
            #print(L, l, )#np.product(L == l, axis=1) != 0)
            mask_l = (np.product(L == l, axis=1) != 0)
            for j,a in enumerate(A_levels):
                result[mask_l * (A == a)] = phat_Y_given_L_A_table[i,j]
        return result
    
    return phat_Y_given_L_A

In [7]:
def fit_phat_L(L):
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    M = L.shape[1]
    L_levels = all_binary_vectors(M)
    phat_L_table = np.zeros((len(L_levels),))

    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        phat_L_table[i] = np.mean(mask_l)
        
    def phat_L(L):
        L = np.reshape(np.array(L), (-1,M)) # force L to be a 2D array
        result = np.zeros((L.shape[0],))
        for i,l in enumerate(L_levels):
            mask_l = (np.product(L == l, axis=1) != 0)
            result += mask_l * phat_L_table[i]
        return result
        
    return phat_L

In [8]:
def standardized_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    L_levels = all_binary_vectors(L.shape[1])
    Y_mean = np.zeros((len(A_levels),))

    phat_L = fit_phat_L(L)
    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    
    for i,a in enumerate(A_levels):
        for l in L_levels:
            Y_mean[i] += (phat_Y_given_L_A(l, a) * phat_L(l))[0]

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [9]:
pd.DataFrame(standardized_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


In [10]:
def outcome_regression_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    Y_mean = np.zeros((len(A_levels),))

    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    
    for i,a in enumerate(A_levels):
        Y_mean[i] += np.mean(phat_Y_given_L_A(L, a))

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [11]:
pd.DataFrame(outcome_regression_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the inverse probability weighted means for each level of A for a given dataset

In [12]:
# Separately tabulate the freqency of A for each possible level of L
# (note: vectorized with respect to samples, so fast but a little ugly)
def fit_phat_A_given_L(L, A):
    A = np.array(A)
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A_levels = [False, True]
    L_levels = all_binary_vectors(L.shape[1])
    phat_A_given_L_table = np.zeros((len(L_levels), len(A_levels)))

    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        P_l = np.mean(mask_l)
        for j,a in enumerate(A_levels):
            phat_A_given_L_table[i,j] = np.mean(mask_l * (A == a))/P_l
            
    def phat_A_given_L(L):
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
        result = np.zeros((L.shape[0], len(A_levels)))
        for i,l in enumerate(L_levels):
            mask_l = (np.product(L == l, axis=1) != 0)
            for j,a in enumerate(A_levels):
                result[:,j] += mask_l * phat_A_given_L_table[i,j]
        return result
        
    return phat_A_given_L

In [ ]:
def ip_weighted_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    phat_A_given_L = fit_phat_A_given_L(L, A)
    Y_mean = [np.mean(Y * (A == a) / phat_A_given_L(L)[:,i]) for i,a in enumerate(A_levels)]

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [ ]:
pd.DataFrame(ip_weighted_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the double-robust means for each level of A for a given dataset

In [15]:
def doubly_robust_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    Y = np.array(Y)
    
    phat_A_given_L = fit_phat_A_given_L(L, A)
    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    f = phat_A_given_L(L)
    
    Y_mean = np.zeros((len(A_levels),))
    for i,a in enumerate(A_levels):
        q = phat_Y_given_L_A(L, a)
        Y_mean[i] = np.mean(q + (A == a) / f[:,i] * (Y - q))

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [16]:
pd.DataFrame(doubly_robust_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the means for each level of A using a marginal structural model (in this case, inverse probability of treatment weighting using logistic marginal models)

In [69]:
# Use a logistic marginal model for L
def fit_marginal_phat_A_given_L(L, A):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)
    
    model = LogisticRegression().fit(L, A)
    
    def phat_A_given_L(L):
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
        return model.predict_proba(L)
        
    return phat_A_given_L

In [250]:
def fit_marginal_phat_P_Y_given_L_A(L, A, Y):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)

    marginal_phat_A_given_L = fit_marginal_phat_A_given_L(L, A)
    #weights = np.ones_like(A) 
    #weights = 1/marginal_phat_A_given_L(L)[range(L.shape[0]),A.astype(int)]
    weights = A/marginal_phat_A_given_L(L)[:,1] + (1-A)/marginal_phat_A_given_L(L)[:,0]
    #X = np.hstack([L, A[:,np.newaxis]])
    X = np.hstack([A[:,np.newaxis]])
    model = LogisticRegression().fit(X, Y, weights)

    def phat_Y_given_L_A(L, A):
        A = np.array(A)
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array        
        #X = np.hstack([L, A[:,np.newaxis]])
        X = np.hstack([A[:,np.newaxis]])
        return model.predict_proba(X)
        
    return phat_Y_given_L_A   

In [251]:
def marginal_structural_model_for_A(L, A, Y):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)
    Y = np.array(Y)

    marginal_phat_P_Y_given_L_A = fit_marginal_phat_P_Y_given_L_A(L, A, Y)
    
    A_levels = [False, True]
    Y_mean = [np.mean(marginal_phat_P_Y_given_L_A(L, np.ones_like(A)*a)[:,1]) for a in A_levels]
    
    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [252]:
pd.DataFrame(marginal_structural_model_for_A(**ch2data))

,A_level,Y_mean
0,False,0.474134
1,True,0.511837


### Calculate the odds ratio using g-estimation

If $g(x) = x$

$$g(\mathbb{E}(Y^a)) - g(\mathbb{E}(Y^{a=0})) = \Psi_0 a + \Psi_L al$$

$$U = \mathbb{E}(Y^{a=0}) = g^{-1}(g(\mathbb{E}(Y^a)) - \Psi_0 a - \Psi_L al)$$


We first define a structural mean model (SMM) with link function  $g(x) = \mathrm{logit}(x)$ parameterized by $\Psi_0$ and $\Psi_L$:

$$\mathrm{logit}\left(\mathbb{E}\left(Y^a|L=l,A=a\right)\right) 
    - \mathrm{logit}\left(\mathbb{E}\left(Y^0|L=l,A=a\right)\right)
    = \gamma(a, l; \Psi_0, \Psi_L),$$
    
where

$$\gamma(a, l; \Psi_0, \Psi_L) = \Psi_0 a + \Psi_L al. $$
    
We can then construct a variable $U(\Psi_0, \Psi_L)$ such that

$$U = \mathrm{expit}\left(\mathrm{logit}\left(\mathbb{E}\left(Y^a\right)\right) - \Psi_0 a - \Psi_L al\right)$$

such that

$$\mathbb{E}\left(U\right )= \mathbb{E}\left(Y^{a=0}\right)$$

$$P(A | L) = expit(B_0 + B_l * L + B_u * U(A, L; \Psi_a, \Psi_L))$$

$$A \perp\perp Y^{a=0} | L$$

### Utilities

A simple utility function for summarizing results for a given model

In [253]:
def summarize_model(data, title='', description=''):
    A = data['A']
    L = np.reshape(np.array(data['L']), (len(A), -1)) # force L to be a 2D array
    Y = data['Y']
    
    display(Markdown('### ' + title))
    display(Markdown(description))
    display(Markdown('#### Sample data'))
    sample_df = pd.concat([
        pd.DataFrame(L).add_prefix("L"), 
        pd.Series(A, name='A'), 
        pd.Series(Y, name='Y')
        ], axis=1)
    display(sample_df.iloc[:12])

    display(Markdown('#### Summary statistics'))
    stats = {
        '$\\bar L$': [np.mean(L[:,i]) for i in range(L.shape[1])],
        '$\\bar A$': np.mean(A),
        '$\\bar Y$': np.mean(Y),
    }
    if 'Y_a_0' in data:
        stats = stats | {
        '$\\bar Y^c_0$': np.mean(data['Y_a_0']),
        '$\\bar Y^c_1$': np.mean(data['Y_a_1']),
        }
    display(pd.DataFrame({'statistic': stats.values()}, index=stats.keys()))
    
    model_names = []
    Y_hat_a_0s = []
    Y_hat_a_1s = []
    
    if 'Y_a_0' in data:
        model_names.append('Underlying counterfactual data')
        Y_hat_a_0s.append(np.mean(data['Y_a_0']))
        Y_hat_a_1s.append(np.mean(data['Y_a_1']))
        

    models = [
        ["Standardized means", standardized_means_for_A],
        ["Outcome regression", outcome_regression_for_A], 
        ["Inverse probability weighted means (IPW)", ip_weighted_means_for_A],
        ["Doubly robust means", doubly_robust_means_for_A],
        ["Marginal structural model (MSM)", marginal_structural_model_for_A]
    ]
    for model_name, model in models:
        model_names.append(model_name)
        result = model(L, A, Y)
        Y_hat_a_0s.append(result['Y_mean'][0])
        Y_hat_a_1s.append(result['Y_mean'][1])
   
    display(Markdown('#### Estimators'))
    display(pd.DataFrame({'$\hat Y^{a=0}$':Y_hat_a_0s, '$\hat Y^{a=1}$':Y_hat_a_1s}, index=model_names))

$\hat Y^{a=0}$

In [254]:
summarize_model(ch2data, 'Analysis of data from chapter 2', 'Validating the summary function with data from chapter 2')

### Analysis of data from chapter 2

Validating the summary function with data from chapter 2

#### Sample data

,L0,A,Y
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,1,1
8,1,0,1
9,1,0,1


#### Summary statistics

,statistic
$\bar L$,[0.6]
$\bar A$,0.65
$\bar Y$,0.5


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Standardized means,0.500000,0.500000
Outcome regression,0.500000,0.500000
Inverse probability weighted means (IPW),0.500000,0.500000
Doubly robust means,0.500000,0.500000
Marginal structural model (MSM),0.474134,0.511837


## Testing with generated data

Start with a simple parameterized model...

In [255]:
def generate_simple_logistic_model_data(N=1000000, P_L=0.5,
                                        beta_A_0=0, beta_A_L=0, 
                                        beta_Y_0=0, beta_Y_L=0, beta_Y_A=0, beta_Y_LA=0,
                                        debug_info=False):
    # reshape and broadcast values into appropriately sized arrays
    P_L = np.array(P_L).reshape((-1,)) # expand P_L to a 2d array if needed
    M = len(P_L)
    if np.isscalar(beta_A_L) == 1: # broadcast to appropriate shape if needed
        beta_A_L = np.array([beta_A_L]*M)
    else:
        beta_A_L = np.array([beta_A_L]).reshape((M,))
    if np.isscalar(beta_Y_L) == 1: # broadcast to appropriate shape if needed
        beta_Y_L = np.array([beta_Y_L]*M)
    else:
        beta_Y_L = np.array([beta_Y_L]).reshape((M,))
    if np.isscalar(beta_Y_LA) == 1: # broadcast to appropriate shape if needed
        beta_Y_LA = np.array([beta_Y_LA]*M)
    else:
        beta_Y_LA = np.array([beta_Y_LA]).reshape((M,))
    
    L = np.random.uniform(size=(N,)+np.array(P_L).shape) < P_L
    P_A_given_l = sps.logistic.cdf(beta_A_0 + L.dot(beta_A_L))
    A = np.random.uniform(size=N) < P_A_given_l
    #P_Y_given_l_a = sps.logistic.cdf(beta_Y_0 + L.dot(beta_Y_L) +
    #                                 A.dot(beta_Y_A) +
    #                                 L.dot(beta_Y_LA)*A
    #                                 )
    #Y = np.random.uniform(size=N) < P_Y_given_l_a
    P_Y_given_l_a_0 = sps.logistic.cdf(beta_Y_0 + L.dot(beta_Y_L) #+
                                     #A.dot(beta_Y_A) +
                                     #L.dot(beta_Y_LA)*A
                                     )
    P_Y_given_l_a_1 = sps.logistic.cdf(beta_Y_0 + L.dot(beta_Y_L) +
                                     beta_Y_A +#A.dot(beta_Y_A) +
                                     L.dot(beta_Y_LA)#L.dot(beta_Y_LA)*A
                                     )
    Y_a_0 = np.random.uniform(size=N) < P_Y_given_l_a_0
    Y_a_1 = np.random.uniform(size=N) < P_Y_given_l_a_1
    Y = Y_a_0 * (1 - A) + Y_a_1 * A
    
    return {
        'L' : L,
        'A' : A,
        'Y' : Y,
        'Y_a_0': Y_a_0,
        'Y_a_1': Y_a_1,
    } | ({
        'P_L' : P_L,
        'P_A_given_l' : P_A_given_l, 
        'P_Y_given_l_a' : P_Y_given_l_a,
    } if debug_info else {})

In [256]:
summarize_model(generate_simple_logistic_model_data(), 'Maximal entropy model', 'No interactions, 1:1 odds for L, A, and Y')

### Maximal entropy model

No interactions, 1:1 odds for L, A, and Y

#### Sample data

,L0,A,Y
0,True,False,1
1,False,False,1
2,False,False,0
3,True,False,0
4,False,False,1
5,False,True,1
6,True,True,1
7,True,False,0
8,False,False,1
9,True,True,1


#### Summary statistics

,statistic
$\bar L$,[0.499665]
$\bar A$,0.500193
$\bar Y$,0.501336
$\bar Y^c_0$,0.500368
$\bar Y^c_1$,0.500721


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.500368,0.500721
Standardized means,0.500755,0.501916
Outcome regression,0.500755,0.501916
Inverse probability weighted means (IPW),0.500755,0.501916
Doubly robust means,0.500755,0.501916
Marginal structural model (MSM),0.500756,0.501919


In [257]:
summarize_model(generate_simple_logistic_model_data(beta_Y_A=-1),
                'Simple randomized protective intervention', 'Assumes no effect from covariate')

### Simple randomized protective intervention

Assumes no effect from covariate

#### Sample data

,L0,A,Y
0,True,True,1
1,False,False,1
2,False,False,1
3,True,True,1
4,False,True,0
5,True,False,1
6,False,True,0
7,False,False,1
8,True,True,0
9,False,True,0


#### Summary statistics

,statistic
$\bar L$,[0.499713]
$\bar A$,0.499984
$\bar Y$,0.383309
$\bar Y^c_0$,0.499704
$\bar Y^c_1$,0.268135


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.499704,0.268135
Standardized means,0.499056,0.267554
Outcome regression,0.499056,0.267554
Inverse probability weighted means (IPW),0.499056,0.267554
Doubly robust means,0.499056,0.267554
Marginal structural model (MSM),0.499056,0.267555


In [258]:
summarize_model(generate_simple_logistic_model_data(beta_Y_A=-1, beta_Y_L=1),
                'Randomized, covariate is risk, intervention is protective',
                '1:1 odds of covariate')

### Randomized, covariate is risk, intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,False,True,0
1,False,False,1
2,True,True,0
3,True,True,0
4,False,False,1
5,True,False,1
6,True,True,0
7,True,True,1
8,True,True,0
9,False,True,0


#### Summary statistics

,statistic
$\bar L$,[0.499671]
$\bar A$,0.499374
$\bar Y$,0.500402
$\bar Y^c_0$,0.615631
$\bar Y^c_1$,0.384663


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.615631,0.384663
Standardized means,0.616096,0.384418
Outcome regression,0.616096,0.384418
Inverse probability weighted means (IPW),0.616096,0.384418
Doubly robust means,0.616096,0.384418
Marginal structural model (MSM),0.616095,0.384419


Note that $Y_\mathrm{mean}$ in the first row corresponds to $P(Y | \mathrm{do}(A=\mathrm{False}))$ and the second row corresponds to $P(Y | \mathrm{do}(A=\mathrm{True}))$.

The first row is $\mathrm{sigmoid}(0\cdot \beta_A + \ldots)$

The second row is $\mathrm{sigmoid}(1 \cdot \beta_A + \ldots)$

We can recover $\beta_A$ by inverting the sigmoid function for each and taking the difference between them:

In [259]:
sp.special.logit(0.383850)-sp.special.logit(0.616879)

-0.9495606173935895

In [260]:
summarize_model(generate_simple_logistic_model_data(beta_A_0=-2, beta_Y_A=-1, beta_Y_L=1),
                'Randomized, covariate is risk, rare intervention is protective',
                '1:1 odds of covariate')

### Randomized, covariate is risk, rare intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,False,False,1
1,True,False,1
2,True,False,1
3,False,True,0
4,False,False,1
5,True,False,1
6,False,False,0
7,False,False,1
8,True,False,1
9,False,False,1


#### Summary statistics

,statistic
$\bar L$,[0.499032]
$\bar A$,0.11906
$\bar Y$,0.587879
$\bar Y^c_0$,0.615191
$\bar Y^c_1$,0.383876


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.615191,0.383876
Standardized means,0.615489,0.383589
Outcome regression,0.615489,0.383589
Inverse probability weighted means (IPW),0.615489,0.383589
Doubly robust means,0.615489,0.383589
Marginal structural model (MSM),0.615488,0.383590


In [261]:
summarize_model(generate_simple_logistic_model_data(P_L=0.75, beta_A_0=-2, beta_Y_A=-1, beta_Y_L=1),
                'Randomized, common covariate is risk, rare intervention is protective',
                '1:1 odds of covariate')

### Randomized, common covariate is risk, rare intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,True,False,1
1,False,False,1
2,True,False,1
3,False,False,1
4,True,False,1
5,True,False,1
6,False,False,0
7,True,False,1
8,True,False,0
9,True,False,0


#### Summary statistics

,statistic
$\bar L$,[0.75035]
$\bar A$,0.118688
$\bar Y$,0.645684
$\bar Y^c_0$,0.673376
$\bar Y^c_1$,0.441312


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.673376,0.441312
Standardized means,0.673505,0.439101
Outcome regression,0.673505,0.439101
Inverse probability weighted means (IPW),0.673505,0.439101
Doubly robust means,0.673505,0.439101
Marginal structural model (MSM),0.673504,0.439102


In [262]:
summarize_model(generate_simple_logistic_model_data(P_L=0.75,
                                                    beta_A_0=-2, beta_A_L=1, 
                                                    beta_Y_A=-1, beta_Y_L=1),
                'Common covariate is risk for disease and intervention, intervention is protective',
                '')

### Common covariate is risk for disease and intervention, intervention is protective

#### Sample data

,L0,A,Y
0,False,False,0
1,False,False,1
2,True,False,1
3,False,True,0
4,True,False,0
5,True,False,1
6,True,True,1
7,False,True,0
8,True,True,0
9,True,False,1


#### Summary statistics

,statistic
$\bar L$,[0.749782]
$\bar A$,0.231412
$\bar Y$,0.620031
$\bar Y^c_0$,0.673026
$\bar Y^c_1$,0.441722


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.673026,0.441722
Standardized means,0.673247,0.443246
Outcome regression,0.673247,0.443246
Inverse probability weighted means (IPW),0.673247,0.443246
Doubly robust means,0.673247,0.443246
Marginal structural model (MSM),0.673246,0.443249


In [263]:
summarize_model(generate_simple_logistic_model_data(P_L=0.5,
                                                    beta_A_0=0, 
                                                    beta_Y_A=1, beta_Y_L=1, beta_Y_LA=-2),
                'Covariate and intervention are risks, but negate eachother',
                '1:1 odds of covariate')

### Covariate and intervention are risks, but negate eachother

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,True,True,0
1,True,False,0
2,True,True,0
3,False,True,1
4,False,False,1
5,True,True,1
6,False,False,0
7,True,True,1
8,True,True,1
9,False,True,1


#### Summary statistics

,statistic
$\bar L$,[0.499846]
$\bar A$,0.50031
$\bar Y$,0.615758
$\bar Y^c_0$,0.615456
$\bar Y^c_1$,0.614982


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.615456,0.614982
Standardized means,0.616062,0.615418
Outcome regression,0.616062,0.615418
Inverse probability weighted means (IPW),0.616062,0.615418
Doubly robust means,0.616062,0.615418
Marginal structural model (MSM),0.616062,0.615418


## Simulations with multi-dimensional $L$

In [264]:
summarize_model(generate_simple_logistic_model_data(P_L=(0.5,0.5)), 
    'Multidimensional maximal entropy model', 'No interactions, 1:1 odds for L, A, and Y')

### Multidimensional maximal entropy model

No interactions, 1:1 odds for L, A, and Y

#### Sample data

,L0,L1,A,Y
0,True,False,True,0
1,False,True,False,1
2,True,True,False,0
3,False,False,True,1
4,False,True,True,1
5,True,True,True,0
6,True,False,True,0
7,False,True,False,0
8,False,False,True,0
9,True,False,True,0


#### Summary statistics

,statistic
$\bar L$,"[0.499484, 0.499811]"
$\bar A$,0.500642
$\bar Y$,0.500205
$\bar Y^c_0$,0.499924
$\bar Y^c_1$,0.500799


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.499924,0.500799
Standardized means,0.499990,0.500417
Outcome regression,0.499990,0.500417
Inverse probability weighted means (IPW),0.499990,0.500417
Doubly robust means,0.499990,0.500417
Marginal structural model (MSM),0.499991,0.500418


In [265]:
summarize_model(generate_simple_logistic_model_data(
    P_L=(0.2,0.5), beta_A_0=-2, beta_A_L=(4,0), beta_Y_L=(2, 1), beta_Y_LA=(-2,2)), 
    'Treatment already targeted',
    'Treatment only benefits rare condition, commonly causes harm. Common risk factor present.')

### Treatment already targeted

Treatment only benefits rare condition, commonly causes harm. Common risk factor present.

#### Sample data

,L0,L1,A,Y
0,False,False,False,0
1,False,False,False,0
2,False,False,False,1
3,False,False,False,1
4,False,True,True,1
5,False,False,False,0
6,False,True,False,1
7,False,False,False,1
8,False,False,False,0
9,False,True,False,0


#### Summary statistics

,statistic
$\bar L$,"[0.199895, 0.500344]"
$\bar A$,0.271639
$\bar Y$,0.652901
$\bar Y^c_0$,0.675723
$\bar Y^c_1$,0.726541


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.675723,0.726541
Standardized means,0.675534,0.726753
Outcome regression,0.675534,0.726753
Inverse probability weighted means (IPW),0.675534,0.726753
Doubly robust means,0.675534,0.726753
Marginal structural model (MSM),0.675526,0.726753


In [266]:
summarize_model(generate_simple_logistic_model_data(
    P_L=(0.5,0.2), beta_A_0=-2, beta_A_L=(0,4), beta_Y_L=(1, 2), beta_Y_LA=(2,-2)), 
    'Treatment already targeted, swapped L values',
    'Treatment only benefits rare condition, commonly causes harm. Common risk factor present.')

### Treatment already targeted, swapped L values

Treatment only benefits rare condition, commonly causes harm. Common risk factor present.

#### Sample data

,L0,L1,A,Y
0,True,False,True,1
1,False,True,True,1
2,False,False,False,0
3,True,False,True,1
4,False,False,False,0
5,False,True,True,0
6,False,False,False,0
7,True,False,False,0
8,False,False,False,0
9,True,False,False,1


#### Summary statistics

,statistic
$\bar L$,"[0.499881, 0.199595]"
$\bar A$,0.271648
$\bar Y$,0.652535
$\bar Y^c_0$,0.675164
$\bar Y^c_1$,0.725938


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$
Underlying counterfactual data,0.675164,0.725938
Standardized means,0.675476,0.724925
Outcome regression,0.675476,0.724925
Inverse probability weighted means (IPW),0.675476,0.724925
Doubly robust means,0.675476,0.724925
Marginal structural model (MSM),0.675456,0.724814


## Notes for next time

- Consider adding g-estimation (Jamie Robbins, different than g-formula, performs well but difficult to understand)
- Consider looking a propensity weighting of general hospital population and population with CAM screens to use model for delirium prediction with patients with CAM screens to predict risk in general population (a censoring problem)
- Consider simulating L to have same range as actual data (or sampled from actual data?) or subset of actual L?  Likely better to wait until new data import pipeline is working.
- (Later) will need to think about feature selection for real model; may want to filter first for features correlated with outcome or intervention, then go through by hand for likely causal relation given domain knowledge.
- Consider replacing existing non-parametric estimators - likely will require trying different models with outcome regression, IPW, and doubly robust (all should match if models are working properly).
- Could consider using using difference between IPW and outcome regression as part of loss function? (unproven - only guarantees that they match)